In [33]:
from PIL import Image
import numpy as np
from numpy import asarray, expand_dims
from os import listdir
from os.path import isdir

In [34]:
#function to load face
def load_face(filename):
    #load file image
    image = Image.open(filename)
    #convert to RGB
    image = image.convert("RGB")
    
    return asarray(image)

In [35]:
#function to load faces from each file
def load_faces(class_dir):
    faces = list()
    #iterate in folder
    for filename in listdir(class_dir):
        path = class_dir + filename
        try:
            faces.append(load_face(path))
        except:
            print("Erro ao carregar a imagem: " + path)
    return faces

In [36]:
#function to load all dataset of each class
def load_all_faces(source_dir):
    X, y = list(), list()
    #iterate by classes
    for class_dir in listdir(source_dir):
        path = source_dir + class_dir + '/'
        
        if not isdir(path):
            pass
        #load faces of the class_dir
        faces = load_faces(path)
        #fill list with class_dir as the label of the face
        labels = [class_dir for _ in range(len(faces))]
        
        #result
        print("Carregado %d faces da classe: %s" %(len(faces), class_dir))
        
        X.extend(faces)
        y.extend(labels)
    return asarray(X),asarray(y)

In [37]:
## Loading all images
trainX, trainy = load_all_faces('./Dataset/Treino/')
valX, valy = load_all_faces('./Dataset/Validacao/')

Carregado 47 faces da classe: Dessa
Carregado 47 faces da classe: Diego
Carregado 50 faces da classe: Diessa
Carregado 7 faces da classe: Dessa
Carregado 7 faces da classe: Diego


In [38]:
from tensorflow.keras.models import load_model

In [39]:
#Load facenet
model = load_model('facenet_keras.h5')
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [40]:
#Generate Embeddings
def get_embedding(model, face_pixels):
    #Standardization (reduzir a magnitude de dados)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    #expand dims -> (1,160,160)
    samples = expand_dims(face_pixels, axis=0)
    
    #predict to generate the embedding
    #[[1,2,...,128],[1,2,...,128]...]
    yhat = model.predict(samples)
    
    return yhat[0]

In [41]:
newTrainX = list()
newValX = list()

In [42]:
#dados treinamento
for face_pixels in trainX:
    newTrainX.append(get_embedding(model, face_pixels))
#dados validacao
for face_pixels in valX:
    newValX.append(get_embedding(model, face_pixels))
    

In [43]:
# embeddings to array list
newTrainX = asarray(newTrainX)
newValX = asarray(newValX)
print(newTrainX.shape,newValX.shape)

(144, 128) (14, 128)


In [44]:
import pandas as pd
#gera dataframe with each embedding
df_train = pd.DataFrame(data=newTrainX)
df_val = pd.DataFrame(data=newValX)

In [45]:
#add the class label to each image
df_train['target'] = trainy
df_val['target'] = valy

In [46]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.646290,-1.687331,2.883557,-1.046404,0.356762,-0.031084,0.818687,1.042676,0.990864,-1.797787,...,-0.620700,0.441091,-0.397414,0.684068,-0.644207,-0.123236,0.243865,-0.861191,-0.262962,Dessa
1,-1.194539,-0.351180,0.847829,-1.458992,-0.670082,0.320437,1.117681,1.531365,-0.245236,-1.541856,...,-0.458893,2.197925,-0.467323,0.565679,-0.843947,-0.442665,-1.723495,-0.829319,-0.915569,Dessa
2,-1.459124,0.350540,1.922037,-1.547778,-0.076800,1.135906,2.339353,1.170720,1.278217,-1.286478,...,-0.758161,1.246184,0.017751,0.110696,0.255546,-0.881180,-0.652305,-1.156251,-1.029011,Dessa
3,-1.397213,-2.582741,1.606574,-0.014072,0.152207,0.110997,1.238310,1.298510,-0.061058,-0.169997,...,-0.850449,2.886897,-0.118972,0.003521,-1.562422,-0.469554,-0.094245,0.848792,-1.233757,Dessa
4,-0.556707,-1.063721,1.920441,-1.446446,-0.390729,0.597911,0.234126,1.192957,1.042021,-1.012844,...,0.516457,0.996243,-0.568146,1.012027,-0.807611,-0.534911,-1.295959,0.443297,0.799314,Dessa


In [47]:
df_val.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.245843,-3.227210,2.465695,-0.917892,-0.273218,0.014268,0.960072,2.046350,0.770749,-1.729095,...,-0.944039,1.675355,-0.282118,0.183822,-0.653568,0.195866,-0.342965,0.134062,0.085601,Dessa
1,-1.820353,-1.927011,2.204384,-1.009596,0.881777,-0.266035,1.083478,0.185461,0.970948,-0.101829,...,-0.239811,1.173024,-0.469987,0.834028,0.029961,-1.558266,0.222502,-1.249710,-0.936888,Dessa
2,-1.396999,-1.730301,0.985809,-0.609296,0.259128,-0.350798,1.171645,1.098546,0.260584,0.165752,...,-2.360200,1.195297,0.235626,-0.853008,-1.222476,-1.174283,0.642346,-0.408757,-1.177594,Dessa
3,-1.636572,-0.296489,1.418785,-1.641847,0.345870,0.003038,1.054012,1.722565,0.826067,-1.871242,...,-0.836204,1.313731,0.584170,0.849724,-0.827995,-1.066870,-0.335423,-0.783863,-0.067064,Dessa
4,-0.927681,-0.953416,3.014898,-2.241177,-0.365648,0.755285,0.518138,1.552790,0.988955,-1.952708,...,-1.770725,1.579803,-0.078469,0.435631,-0.343946,-0.115035,-0.751708,-1.119460,-0.966538,Dessa


In [48]:
df_train.to_csv('faces_train_embeddings.csv')
df_val.to_csv('faces_validation_embeddings.csv')